### Required Libraries

In [1]:
import numpy as np
import os,cv2,math,glob,random
import scipy.io as sio
from math import cos, sin
from pathlib import Path
import pandas as pd
import mediapipe
import warnings
warnings.filterwarnings('ignore')
import mediapipe as mp
import matplotlib.pyplot as plt
import cv2 
from Preprocessing_Utility_Functions import *



### Preparing and collect Data

In [2]:
# define the path to the folder
folder_path = r"C:\Users\mohamed elsayed\Desktop\Pose Estimation\AFLW2000"
# define data structures to store the extracted data
xy = {}
yaw = []
pitch = []
roll = []
nose = {}

# Loop through all files in the folder
for imgname in os.listdir(folder_path):
    if imgname.lower().endswith(('.jpg', '.mat')):
        if imgname.lower().endswith('.jpg'):
            process_jpg(imgname, xy,nose)
        elif imgname.lower().endswith('.mat'):
            process_mat(imgname, roll, pitch, yaw)

# Create a DataFrame from the extracted xy
df = pd.DataFrame.from_dict(xy, orient='index')
# change the column names to x0, y0, x1, y1, ...
df.columns = [f'x{i//2}' if i % 2 == 0 else f'y{i//2}' for i in range(df.shape[1])]
# add roll, pitch, and yaw to the DataFrame
df["roll"] = roll  
df["pitch"] = pitch
df["yaw"] = yaw


In [3]:
#show the first  rows of the dataframe
df

,x0,y0,x1,y1,x2,y2,x3,y3,x4,y4,...,y464,x465,y465,x466,y466,x467,y467,roll,pitch,yaw
image00002.jpg,-0.023765,0.318035,-0.006607,0.089624,-0.003660,0.126111,-0.052435,-0.284172,0.0,0.0,...,-0.621766,0.246334,-0.585165,0.847953,-0.725765,0.898422,-0.761265,-0.399231,0.018227,0.085676
image00004.jpg,0.135390,0.342489,0.007973,0.080386,0.173977,0.176596,0.106425,-0.167740,0.0,0.0,...,-0.343096,0.446485,-0.326326,0.887145,-0.403468,0.937679,-0.429012,0.470065,1.189533,0.300959
image00006.jpg,-0.034451,0.295173,-0.001236,0.081678,-0.029573,0.134064,-0.097803,-0.205510,0.0,0.0,...,-0.441704,0.067555,-0.418426,0.488717,-0.515062,0.528595,-0.545855,-0.184650,0.881137,-0.236852
image00008.jpg,0.206938,0.278244,0.038110,0.079997,0.136178,0.103737,-0.085015,-0.232270,0.0,0.0,...,-0.623308,0.141488,-0.583820,0.656938,-0.893207,0.691559,-0.947129,-0.175379,0.299208,-0.373374
image00010.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.882169,1.198004,-1.033374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
image04358.jpg,-0.056898,0.364123,-0.013462,0.097607,-0.029582,0.156435,-0.064382,-0.279807,0.0,0.0,...,-0.582368,0.225926,-0.553400,0.822239,-0.653323,0.873485,-0.689914,-0.197102,-0.070430,0.105118
image04363.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.232617,-1.418751,0.175960
image04364.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.447955,1.431267,-1.509418
image04365.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.420465,-1.191176,0.451515


### drop missing values 

In [ ]:
# drop rows with missing values
df.dropna(inplace=True)

### save data in csv 

In [15]:
# save the DataFrame to a CSV file
df.to_csv('face_landmarks.csv', index=False)